#### Returns 5 random frames for a certain pose.

In [2]:
import random


# pose file (output from matlab file)
fileName = r"D:\\Research Lab\\MD Sim\\completed_md\\data_storage\\WT_G196F\\poses_v2\\G196F_topLeft-r23.txt"


# open up file and extract frames
frameDict=[] # input: [[run_type],[run_number],[frame number],[dcd_number]]

runType = fileName.split("\\")[-1].split("_")[0]
with open(fileName, "r") as f:
    text = f.read()
    textSplit = text.split("\n")
    runNum = ""
    for line in textSplit:
        if line.startswith("run"):
            runNum = line
            continue
        nums = line.split(" ")
        if(len(nums) > 0):
            for index,num in enumerate(nums):
                if(num != ""):
                    frameDict.append([runType,runNum,num,str(index)])


#pick 5 tandom and return
# note: VMD starts coutning frames at 0
print("Run Type   Run Number   OG Frame Number   DCD Pose Frame Number")
for number in range(1,6,1):
    op = random.choice(frameDict)
    frameDict.remove(op)
    print(",".join(op))




Run Type   Run Number   OG Frame Number   DCD Pose Frame Number
G196F,run2,1591,79
G196F,run2,1360,56
G196F,run2,1321,47
G196F,run2,1603,82
G196F,run2,1285,41


#### Creates Nicely Formatted Interaction Data csv

In [56]:
    
output = output.replace("\\", "/")
output = output.replace('"', "")

splitText = inputed.split("\n")

allInteractions = []
for line in splitText:
    splitLine = line.split("\t")
    # for every single line, split along \t:
    op = []

    if(len(splitLine) < 5):
        continue

    if(splitLine[7].split(":")[1] == "FAD1" and splitLine[9].split(":")[1] == "FAD1"):
        continue

    #if a h-bond
    if(splitLine[5] == "Hydrogen Bond"):
        #1: type of bond
        if(splitLine[6] == "Conventional Hydrogen Bond"):
            op.append("H-bond")
        elif(splitLine[6] == "Carbon Hydrogen Bond"):
            op.append("Carbon H-bond")
        elif(splitLine[6] == "Pi-Donor Hydrogen Bond"):
            op.append("Pi-donor H-bond")

        #2/3: donor / acceptor
        if(splitLine[8] == "H-Donor"):
            op.append(":".join(splitLine[7].split(":")[1:]))
            op.append(":".join(splitLine[9].split(":")[1:]))
        elif(splitLine[8] == "H-Acceptor"):
            op.append(":".join(splitLine[9].split(":")[1:]))
            op.append(":".join(splitLine[7].split(":")[1:]))
            
        op.append(splitLine[4]) #4: distance
        op.append(splitLine[11]) #5: angle DHA
        op.append(splitLine[12]) #6: angle HAY
    
    
    # if electrostatic
    elif(splitLine[5] == "Electrostatic"):
        op.append(splitLine[6]) #1: type of bond
        op.append(":".join(splitLine[7].split(":")[1:])) #2: positive
        op.append(":".join(splitLine[9].split(":")[1:])) #3: negative / pi
        op.append(splitLine[4]) #4: distance
        if(len(splitLine) > 14):
            op.append(splitLine[13]) #5: theta
    
    
    # if hydrophobic
    elif(splitLine[5] == "Hydrophobic"):
        op.append(splitLine[6]) #1: type of bond
        op.append(":".join(splitLine[7].split(":")[1:])) #2: type of molecule 1
        op.append(":".join(splitLine[9].split(":")[1:])) #3: type of molecule 2
        op.append(splitLine[4]) #4: distance
    else:
        op.append("NOT ORGANIZED TYPE")
        op.append(splitLine[6]) #1: type of bond
        op.append(":".join(splitLine[7].split(":")[1:])) #2: type of molecule 1
        op.append(":".join(splitLine[9].split(":")[1:])) #3: type of molecule 2
        op.append(splitLine[4]) #4: distance 
    
    # add stuff to front
    # get the non FAD residue
    if(op[1].startswith("FAD")):
        work = op[2]
    else:
        work = op[1]

    # repalce with 
    amino_acid_codes = {'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 'GLN': 'Q', 'GLU': 'E', 'GLY': 'G', 'HIS': 'H','HSD': 'H', 'HSE': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V'}
    newName = amino_acid_codes[work[0:3]] + work[3:].split(":")[0]
    op.insert(1, " ")
    op.insert(1, newName)

    
    allInteractions.append(op)

prev = ""
allInteractionsHeader = []
#allInteractions.sort(key=lambda x: x[0].split(" ")[-1])

for op in allInteractions:
    if(not op[0].split(" ")[-1] == prev):
        if(op[0].split(" ")[-1] == "H-bond"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "H-Donor", "H-Acceptor", "Distance", "Angle DHA", "Angle HAY"])
        
        elif(op[0] == "Pi-Cation"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "Positive", "Pi", "Distance", "Theta"])

        elif(op[0] == "Attractive Charge"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "Positive", "Negative", "Distance"])
        
        elif(op[0] == "Alkyl"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "Alkyl 1", "Alkyl 2", "Distance"])
        
        elif(op[0] == "Pi-Alkyl"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "Pi", "Alkyl", "Distance"])
    
        elif(op[0] == "Pi-Pi Stacked"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "Pi", "Pi", "Distance"])
        
        elif(op[0] == "Pi-Sigma"):
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["Type", "Res", " ", "Sigma", "Pi", "Distance"])

        else:
            allInteractionsHeader.append([])
            allInteractionsHeader.append(["UNKOWN TYPE", "Res", " ", "", "", ""])
        
        prev = op[0].split(" ")[-1]
    
    allInteractionsHeader.append(op)



with open(output, "w") as f:
    for interaction in allInteractionsHeader[1:]:
        f.write(",".join(interaction))
        f.write("\n")


In [55]:

output = r"D:\Research Lab\MD Sim\completed_md\pose_dcds\WT_bottomRightOther_r1\r1_f1818.csv"

inputed = """F:FAD1:P2 - A:ASP210:OD1	Yes	255 150 0	Ligand Non-bond Monitor	5.59265	Electrostatic	Attractive Charge	F:FAD1:P2	Positive	A:ASP210:OD1	Negative						
A:ARG157:HN - F:FAD1:O5	Yes	0 255 0	Ligand Non-bond Monitor	2.38595	Hydrogen Bond	Conventional Hydrogen Bond	A:ARG157:HN	H-Donor	F:FAD1:O5	H-Acceptor	156.513	94.413				
A:GLY158:HN - F:FAD1:O7	Yes	0 255 0	Ligand Non-bond Monitor	2.50688	Hydrogen Bond	Conventional Hydrogen Bond	A:GLY158:HN	H-Donor	F:FAD1:O7	H-Acceptor	127.333	122.089				
A:ALA195:HN - F:FAD1:O2	Yes	0 255 0	Ligand Non-bond Monitor	1.70823	Hydrogen Bond	Conventional Hydrogen Bond	A:ALA195:HN	H-Donor	F:FAD1:O2	H-Acceptor	174.133	141.306				
A:LYS217:HZ2 - F:FAD1:O15	Yes	0 255 0	Ligand Non-bond Monitor	2.33698	Hydrogen Bond	Conventional Hydrogen Bond	A:LYS217:HZ2	H-Donor	F:FAD1:O15	H-Acceptor	118.98	108.721				
F:FAD1:H1 - A:ALA155:O	Yes	0 255 0	Ligand Non-bond Monitor	1.78789	Hydrogen Bond	Conventional Hydrogen Bond	F:FAD1:H1	H-Donor	A:ALA155:O	H-Acceptor	168.101	146.907				
F:FAD1:H13 - A:ASP159:OD1	Yes	0 255 0	Ligand Non-bond Monitor	2.18205	Hydrogen Bond	Conventional Hydrogen Bond	F:FAD1:H13	H-Donor	A:ASP159:OD1	H-Acceptor	140.139	117.74				
F:FAD1:H16 - F:FAD1:O14	Yes	0 255 0	Ligand Non-bond Monitor	1.8317	Hydrogen Bond	Conventional Hydrogen Bond	F:FAD1:H16	H-Donor	F:FAD1:O14	H-Acceptor	155.843	112.727				
F:FAD1:H20 - A:ASP210:OD1	Yes	0 255 0	Ligand Non-bond Monitor	1.6578	Hydrogen Bond	Conventional Hydrogen Bond	F:FAD1:H20	H-Donor	A:ASP210:OD1	H-Acceptor	163.441	130.514				
A:VAL194:HA - F:FAD1:O2	Yes	220 255 220	Ligand Non-bond Monitor	2.80762	Hydrogen Bond	Carbon Hydrogen Bond	A:VAL194:HA	H-Donor	F:FAD1:O2	H-Acceptor	123.951	160.223				
A:LYS217:HE2 - F:FAD1:O15	Yes	220 255 220	Ligand Non-bond Monitor	2.4961	Hydrogen Bond	Carbon Hydrogen Bond	A:LYS217:HE2	H-Donor	F:FAD1:O15	H-Acceptor	117.414	111.393				
F:FAD1:H18 - F:FAD1:N4	Yes	220 255 220	Ligand Non-bond Monitor	2.93324	Hydrogen Bond	Carbon Hydrogen Bond	F:FAD1:H18	H-Donor	F:FAD1:N4	H-Acceptor	123.011	101.304				
F:FAD1:H22 - A:ASP210:OD1	Yes	220 255 220	Ligand Non-bond Monitor	2.7419	Hydrogen Bond	Carbon Hydrogen Bond	F:FAD1:H22	H-Donor	A:ASP210:OD1	H-Acceptor	137.716	140.301				
A:HSD213 - F:FAD1	Yes	255 100 200	Ligand Non-bond Monitor	4.88647	Hydrophobic	Pi-Pi Stacked	A:HSD213	Pi-Orbitals	F:FAD1	Pi-Orbitals			38.425	28.643	14.269	4.056
A:HSD213 - F:FAD1	Yes	255 100 200	Ligand Non-bond Monitor	4.58655	Hydrophobic	Pi-Pi Stacked	A:HSD213	Pi-Orbitals	F:FAD1	Pi-Orbitals			15.477	19.513	20.989	4.14
A:ALA195 - F:FAD1:C21	Yes	255 200 255	Ligand Non-bond Monitor	3.99832	Hydrophobic	Alkyl	A:ALA195	Alkyl	F:FAD1:C21	Alkyl						
F:FAD1:C21 - A:MET154	Yes	255 200 255	Ligand Non-bond Monitor	5.41392	Hydrophobic	Alkyl	F:FAD1:C21	Alkyl	A:MET154	Alkyl						
A:TYR197 - F:FAD1:C24	Yes	255 200 255	Ligand Non-bond Monitor	5.24235	Hydrophobic	Pi-Alkyl	A:TYR197	Pi-Orbitals	F:FAD1:C24	Alkyl						
F:FAD1 - A:LEU271	Yes	255 200 255	Ligand Non-bond Monitor	5.47361	Hydrophobic	Pi-Alkyl	F:FAD1	Pi-Orbitals	A:LEU271	Alkyl						
F:FAD1 - A:VAL176	Yes	255 200 255	Ligand Non-bond Monitor	5.03776	Hydrophobic	Pi-Alkyl	F:FAD1	Pi-Orbitals	A:VAL176	Alkyl																																																	
"""

#### Creates The Interaction Table

In [54]:
import os
opFile = r"D:\Research Lab\MD Sim\completed_md\pose_dcds\iteraction_table_WT.csv"
frameName = "B3-5 (r1_f1827)"

# read in data from discovery studio (residue name and interactions)
output = output.replace('"', "")
splitText = inputed.split("\n")

allInteractions = {}
for line in splitText:
    splitLine = line.split("\t")
    
    if(len(splitLine) < 5):
        continue
    if(splitLine[7].split(":")[1] == "FAD1" and splitLine[9].split(":")[1] == "FAD1"):
        continue
    if(not splitLine[7].split(":")[1] == "FAD1" and not splitLine[9].split(":")[1] == "FAD1"):
        continue
    
    # get res name
    names = splitLine[0].split(" - ")
    for name in names:
        name = name.split(":")[1]
        if(not name.startswith("FAD")):
            res = name
    
    if(res not in list(allInteractions.keys())):
       allInteractions[res] = []

    # get interaction type
    if(splitLine[5] == "Hydrogen Bond"):
        if(splitLine[6] == "Conventional Hydrogen Bond"):
            allInteractions[res].append("H")
    elif(splitLine[5] == "Electrostatic"):
        allInteractions[res].append("E")
    elif(splitLine[5] == "Hydrophobic"):
        allInteractions[res].append("P")
    
    if(len(allInteractions[res]) == 0):
        del allInteractions[res]

for (key, value) in allInteractions.items():
    value.sort()
    allInteractions[key] = list(set(value))


# read in the interaction table sheet
if(not os.path.exists(opFile)):
    with open(opFile, "w") as f:
        f.write("Resids")

with open(opFile, "r") as f:
    allText = f.read()

lineSplitText = allText.split("\n")
wordSplit = []
for line in lineSplitText:
    spLine = line.split(",")
    if(len(spLine) > 0):
        wordSplit.append(spLine)

# add in new data to table
wordSplit[0].append(frameName)
for ind, res in enumerate(wordSplit[1:]):
    found = False
    for (key, value) in allInteractions.items():
        if(res[0] == key):
            wordSplit[ind+1].append(" ".join(value))
            found = True
            break
    if(not found):
        wordSplit[ind+1].append("")
    else:
        del allInteractions[key]

for (key, value) in allInteractions.items():
    wordSplit.append([key])
    spaces = len(wordSplit[0]) - 2
    for s in range(spaces):
        wordSplit[-1].append("")
    wordSplit[-1].append(" ".join(value))

#write table
with open(opFile, "w") as f:
    for ind, line in enumerate(wordSplit):
        f.write(",".join(line))
        if(ind < len(wordSplit)-1):
            f.write("\n")

    


print(wordSplit)


[['', 'B1-1 (r2_f1315)', 'B1-2 (r2_f1360)', 'B1-3 (r2_f1670)', 'B1-4 (r2_f1729)', 'B1-5 (r2_f1798)', 'B2-1 (r2_f235)', 'B2-2 (r2_f413)', 'B2-3 (r2_f490)', 'B2-4 (r2_f1472)', 'B2-5 (r2_f1512)', 'B3-1 (r1_f709)', 'B3-2 (r1_f1217)', 'B3-3 (r1_f1617)', 'B3-4 (r1_f1818)', 'B3-5 (r1_f1827)'], ['THR94', 'H', 'H', '', '', '', '', '', '', '', 'H', '', '', '', '', ''], ['TRP95', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', '', '', '', '', ''], ['ARG157', 'H P', 'H P', 'H P', 'H', 'H P', '', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H'], ['GLY158', 'H', 'H', '', 'H', 'H', '', '', '', '', '', 'H', 'H', 'H', 'H', 'H'], ['ASP159', 'H', 'H', 'H', 'H', 'H', '', '', '', '', '', 'H', 'H', 'H', 'H', 'H'], ['TYR197', 'H', 'H P', 'P', 'P', 'H P', 'H P', 'P', 'P', 'P', 'H P', 'P', '', 'P', 'P', 'P'], ['ASP210', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H E', 'H', 'H E', 'H E', 'H'], ['ASN173', 'H', 'H', 'H', 'H', 'H', '', '', '', '', '', '', '', '', '', ''], ['LEU156', 'P', 'P', 'P', 'P', 'P', 